# Zalando scraper

In [132]:
## GOOD TO KNOW 

# website = https://zalando.nl/pre-owned-mode-heren/index.html
# article class = _0mW-4D _0xLoFW JT3_zV mo6ZnF _78xIQ- U4aOaA
# figure class = heWLCX
# tag product brand names = <span> 
# inspecting a product gives us: <a class="_LM JT3_zV CKDt_l CKDt_l LyRfpJ"href="https://www.zalando.nl/g-star-overhemd-blue-zir01hbly-001.html" rel="">


In [13]:
# Import packages
from bs4 import BeautifulSoup
import requests
from time import sleep
from datetime import datetime
import csv
import pandas as pd

In [14]:
import requests
from bs4 import BeautifulSoup

#debug = {'verbose': sys.stderr}
user_agent = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edg/98.0.1108.62'}  



In [15]:
# Create a function to collect all page urls
base_url = "https://www.zalando.nl/pre-owned-mode-heren/?p="
counter = 1

# Now create the function
def generate_page_urls(base_url, num_pages):
    page_urls = []
    
    for counter in range (1, num_pages + 1):
        full_url = base_url + str(counter) + "&order=activation_date"
        page_urls.append(full_url)
    
    return page_urls

In [16]:
import selenium.webdriver

driver = selenium.webdriver.Chrome()

In [17]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

print("done")

/bin/bash: apt-get: command not found
The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

done


In [ ]:
driver.get("https://www.zalando.nl/pre-owned-mode-heren/")

print("done")

In [ ]:
from bs4 import BeautifulSoup
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

# Create a function to collect the product_urls
def create_product_urls(page_urls):
    product_urls = []
    cnt=1
    for page_url in page_urls:
        cnt+=1
        res = requests.get(page_url, headers = user_agent)
        soup = BeautifulSoup(res.text, "html.parser")
        f=open('website_'+str(cnt)+'.html', 'w', encoding='utf-8')
        f.write(res.text)
        f.close()
        products = soup.find_all("div", class_= "DT5BTM w8MdNG cYylcv QylWsg _75qWlu iOzucJ JT3_zV DvypSJ")       
        print(len(products))
        sleep(1)

        for product in products:
            try:
                product_url = product.find("a").attrs["href"]
                print(product_url)
                product_urls.append(product_url)
            except:
                print("dit is geen product")
                
    return product_urls

In [ ]:
product_urls = create_product_urls(page_urls)
len(product_urls)

In [ ]:
for _ in range(5):
    res = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(res, "html.parser")
    
     # Scroll down the bottom of the page         
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    
    # pause for 5 seconds
    sleep(1)
        
    return product_urls

In [38]:
# Create a function to collect all page urls
base_url = "https://www.zalando.nl/pre-owned-mode-heren/?p="
counter = 1

# Now create the function
def generate_page_urls(base_url, num_pages):
    page_urls = []
    
    for counter in range (1, num_pages + 1):
        full_url = base_url + str(counter) + "&order=activation_date"
        page_urls.append(full_url)
    
    return page_urls

In [45]:
page_urls = generate_page_urls(base_url, 2) 

generate_page_urls("https://www.zalando.nl/pre-owned-mode-heren/?p=", 20)

['https://www.zalando.nl/pre-owned-mode-heren/?p=1&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=2&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=3&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=4&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=5&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=6&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=7&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=8&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=9&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=10&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=11&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=12&order=activation_date',
 'https://www.zalando.nl/pre-owned-mode-heren/?p=13&order=activation_date',
 'https://www.zalando

In [46]:
import requests
from bs4 import BeautifulSoup

# Create a function to collect the product_urls
def create_product_urls(page_urls):
    product_urls = []
    cnt=1
    for page_url in page_urls:
        cnt+=1
        res = requests.get(page_url, headers = user_agent)
        soup = BeautifulSoup(res.text, "html.parser")
        f=open('website_'+str(cnt)+'.html', 'w', encoding='utf-8')
        f.write(res.text)
        f.close()
        products = soup.find_all("div", class_= "DT5BTM w8MdNG cYylcv QylWsg _75qWlu iOzucJ JT3_zV DvypSJ")       
        print(len(products))
        sleep(1)

        for product in products:
            try:
                product_url = product.find("a").attrs["href"]
                #print(product_url)
                product_urls.append(product_url)
            except:
                print("dit is geen product")
                    
    return product_urls

In [34]:
product_urls = create_product_urls(page_urls)
len(product_urls)

NameError: name 'user_agent' is not defined

In [26]:
import requests
from bs4 import BeautifulSoup

# scraping information from product pages

def get_product_pages(products):
    product_pages = []
    
    for url in product_urls: 
        res = requests.get(url, headers = user_agent)
        soup = BeautifulSoup(res.text, "html.parser")
        attributes = soup.find_all(class_="DT5BTM VHXqc_ rceRmQ _4NtqZU mIlIve")
        delivery = soup.find(class_="PBR5VH _0xLoFW EJ4MLB")
        
        for attribute in attributes:    
            product = attribute.find("h3", class_="mt1kvu ka2E9k uMhVZi FxZV-M SZKKsK pVrzNP _5Yd-hZ").text
            category = attribute.find("span", class_="EKabf7 R_QwOV").text
            price = attribute.find("span", class_="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP")
            size = attribute.find("span", class_="_7Cm1F9 ka2E9k uMhVZi dgII7d RYghuO pVrzNP").text
            delivery_time = delivery.find("span", class_="u-6V88 ka2E9k uMhVZi dgII7d RYghuO pVrzNP").text
            delivery_time = delivery_time.replace("werkdagen", "")
            delivery_time = delivery_time.replace(" ", "")
            product_pages.append({"url": url,
                                 "brand": product,
                                 "category": category,
                                 "price": price,
                                 "size": size,
                                 "delivery time": delivery_time})
   
    return product_pages
print("done")

done


In [140]:
product_attributes = get_product_pages(product_urls)
product_attributes

[{'url': 'https://www.zalando.nl/g-star-trui-beige-zir01l3fd-001.html',
  'brand': 'G-Star',
  'category': 'Trui',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 16,90</span>,
  'size': 'L',
  'delivery time': '3-6'},
 {'url': 'https://www.zalando.nl/nike-sportswear-sweater-black-zir01l1dd-001.html',
  'brand': 'Nike Sportswear',
  'category': 'Sweater',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 17,90</span>,
  'size': 'XL',
  'delivery time': '3-6'},
 {'url': 'https://www.zalando.nl/polo-ralph-lauren-trui-beige-zir01l0mx-001.html',
  'brand': 'Polo Ralph Lauren',
  'category': 'Trui',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 34,90</span>,
  'size': 'S',
  'delivery time': '3-6'},
 {'url': 'https://www.zalando.nl/jack-and-jones-sweater-black-zir01kr1e-001.html',
  'brand': 'Jack & Jones',
  'category': 'Hoodie',
  'price': <span class="uqkIZw ka2E9k uMhVZi FxZV-M RYghuO pVrzNP">€ 15,90</span>,
  'size': 'L',
 

In [141]:
from datetime import datetime

now = datetime.now()
print(now)

2022-03-14 16:33:58.072194


In [142]:
import csv 

with open("product_description_pre_owned.csv", "w", encoding = 'utf-8', newline='') as csv_file: 
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["url", "brand", "category", "price", "size", "delivery time", "now"])
    now = datetime.now()
    for product in product_attributes: 
        writer.writerow([product['url'], product['brand'], product['category'], product['price'], product['size'], product['delivery time'], now])
print('done!')

PermissionError: [Errno 13] Permission denied: 'product_description_pre_owned.csv'